In [1]:
#Generate vector embedding using tokenizer

In [2]:
import pandas as pd
import numpy as np
import string
from collections import Counter
from sklearn.model_selection import train_test_split
import csv
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from google.colab import files
import io

In [3]:
embedding_dim = 100
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_portion = .8

In [4]:
uploaded = files.upload()

Saving test_features.csv to test_features.csv
Saving train_features.csv to train_features.csv
Saving train_labels.csv to train_labels.csv


In [5]:
# turn a seq into clean tokens
def clean_seq(filename):
    seq_df = pd.read_csv(filename)
    sequences = seq_df['dna'].str.replace('[^ACGT]','N')
    for i in range(len(sequences)):
        sequences[i] = sequences[i].ljust(1058,'N')
    sequences_array = np.array(sequences)
    
    seq_list_truncated = []
    for i in range(len(sequences_array)):
        list_seq = list(sequences_array[i])
        del list_seq[650:]
        seq_list_truncated.append(''.join(list_seq))
    
    #print(np.array(seq_list_truncated).shape)
    
    token_list = []
    
    for i in range(len(seq_list_truncated)):
        List = [seq_list_truncated[i][m:m+4] for m in range(len(seq_list_truncated[i])-3)]
        token_list.append(List)
        
    return token_list

# load sequences and create token list
filename = "train_features.csv"
token_list = clean_seq(filename)

def load_labels(filename):
    df = pd.read_csv(filename)
    labels = df['labels']
    return labels

#load labels
filename = 'train_labels.csv'
labels = load_labels(filename)
softmax_layer = len(set(labels))

In [6]:
train_size = int(len(token_list) * training_portion)

train_seqs = token_list[:train_size]
train_labels = labels[:train_size]

validation_seqs = token_list[train_size:]
validation_labels = labels[train_size:]

In [7]:
print(np.array(token_list).shape)

print("Number of Unique label Ids:", len(set(labels)))
print(train_size)
print(train_seqs[0])
print(train_seqs[1])
print(train_seqs[10])
print(len(train_labels))
print(len(validation_seqs))
print(len(validation_labels))

(12906, 647)
Number of Unique label Ids: 1202
10324
['AACA', 'ACAT', 'CATT', 'ATTA', 'TTAT', 'TATA', 'ATAC', 'TACT', 'ACTT', 'CTTT', 'TTTA', 'TTAT', 'TATT', 'ATTT', 'TTTT', 'TTTC', 'TTCG', 'TCGG', 'CGGA', 'GGAG', 'GAGC', 'AGCA', 'GCAT', 'CATG', 'ATGA', 'TGAT', 'GATC', 'ATCA', 'TCAG', 'CAGG', 'AGGA', 'GGAA', 'GAAT', 'AATA', 'ATAG', 'TAGT', 'AGTA', 'GTAG', 'TAGG', 'AGGA', 'GGAA', 'GAAC', 'AACT', 'ACTT', 'CTTC', 'TTCA', 'TCAT', 'CATT', 'ATTA', 'TTAA', 'TAAG', 'AAGT', 'AGTA', 'GTAT', 'TATA', 'ATAT', 'TATT', 'ATTA', 'TTAA', 'TAAT', 'AATT', 'ATTC', 'TTCG', 'TCGA', 'CGAG', 'GAGC', 'AGCT', 'GCTG', 'CTGA', 'TGAA', 'GAAC', 'AACT', 'ACTA', 'CTAG', 'TAGG', 'AGGA', 'GGAA', 'GAAA', 'AAAT', 'AATC', 'ATCC', 'TCCT', 'CCTG', 'CTGG', 'TGGA', 'GGAG', 'GAGC', 'AGCA', 'GCAT', 'CATT', 'ATTA', 'TTAA', 'TAAT', 'AATT', 'ATTG', 'TTGG', 'TGGA', 'GGAG', 'GAGA', 'AGAT', 'GATG', 'ATGA', 'TGAC', 'GACC', 'ACCA', 'CCAA', 'CAAA', 'AAAT', 'AATT', 'ATTT', 'TTTA', 'TTAT', 'TATA', 'ATAA', 'TAAT', 'AATG', 'ATGT', 'TGTA', 'GT

In [8]:
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(train_seqs)
word_index = tokenizer.word_index

vocab_size = len(word_index) + 1

train_sequences = tokenizer.texts_to_sequences(train_seqs)
train_padded = pad_sequences(train_sequences, padding=padding_type)

max_length = len(train_padded[0])

print(train_padded.shape)

print(len(train_sequences[0]))
print(len(train_padded[0]))

print(len(train_sequences[1]))
print(len(train_padded[1]))

print(len(train_sequences[10]))
print(len(train_padded[10]))

(10324, 647)
647
647
647
647
647
647


In [9]:
validation_sequences = tokenizer.texts_to_sequences(validation_seqs)
validation_padded = pad_sequences(validation_sequences, padding=padding_type, maxlen=max_length)

print(len(validation_sequences))
print(validation_padded.shape)

2582
(2582, 647)


In [10]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(labels)
label_seq = le.transform(labels)

output = []
for v in label_seq:
    output.append([v])

print(output)
training_label_seq = np.array(output[:train_size])
validation_label_seq = np.array(output[train_size:])

[[31], [626], [1163], [30], [461], [450], [1164], [853], [626], [553], [780], [722], [643], [895], [44], [216], [364], [358], [1115], [1055], [384], [1012], [473], [372], [198], [858], [786], [248], [505], [220], [1085], [331], [647], [998], [1185], [714], [900], [152], [10], [614], [372], [370], [507], [321], [1099], [114], [213], [982], [15], [929], [734], [424], [540], [176], [226], [1161], [396], [550], [284], [680], [205], [586], [16], [462], [905], [372], [747], [1148], [641], [53], [822], [1035], [1054], [30], [673], [92], [1150], [461], [514], [959], [366], [444], [871], [848], [391], [1083], [4], [1176], [311], [881], [568], [513], [677], [11], [757], [1142], [997], [1166], [995], [747], [1162], [664], [32], [747], [643], [409], [734], [677], [67], [195], [599], [799], [416], [750], [877], [118], [793], [572], [532], [210], [555], [83], [617], [853], [479], [328], [693], [1200], [1200], [308], [584], [1150], [1172], [556], [468], [380], [524], [835], [686], [553], [809], [105]

In [16]:
#Deep Learning model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(100, dropout=0.2, recurrent_dropout=0, recurrent_activation = 'sigmoid'), #LSTM layer with memory 100
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(1000, activation='relu'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(softmax_layer, activation='softmax')
])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

history = model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 647, 100)          61200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 647, 100)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
batch_normalization_2 (Batch (None, 100)               400       
_________________________________________________________________
flatten_1 (Flatten)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1000)              101000    
_________________________________________________________________
dropout_3 (Dropout)          (None, 1000)             

In [14]:
num_epochs = 30

# Create a callback that saves the model's weights
#https://jonathan-hui.medium.com/tensorflow-save-restore-model-75a1e6d3b9a6
#https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint
callbacks = [
        tf.keras.callbacks.ModelCheckpoint(
        filepath="mymodel_{epoch}", save_weights_only=True,
        save_best_only = True, monitor = "val_loss", verbose =1,)]

model.fit(train_padded, training_label_seq, epochs=num_epochs, callbacks=callbacks, validation_data=(validation_padded, validation_label_seq), verbose=2)

Epoch 1/30
323/323 - 9s - loss: 1.1459 - accuracy: 0.7469 - val_loss: 1.3762 - val_accuracy: 0.7339

Epoch 00001: val_loss improved from inf to 1.37617, saving model to mymodel_1
Epoch 2/30
323/323 - 10s - loss: 0.9228 - accuracy: 0.7997 - val_loss: 1.9010 - val_accuracy: 0.6379

Epoch 00002: val_loss did not improve from 1.37617
Epoch 3/30
323/323 - 9s - loss: 0.8055 - accuracy: 0.8215 - val_loss: 1.0601 - val_accuracy: 0.8102

Epoch 00003: val_loss improved from 1.37617 to 1.06010, saving model to mymodel_3
Epoch 4/30
323/323 - 9s - loss: 0.7036 - accuracy: 0.8452 - val_loss: 1.0794 - val_accuracy: 0.8091

Epoch 00004: val_loss did not improve from 1.06010
Epoch 5/30
323/323 - 9s - loss: 0.6238 - accuracy: 0.8630 - val_loss: 1.2520 - val_accuracy: 0.7754

Epoch 00005: val_loss did not improve from 1.06010
Epoch 6/30
323/323 - 9s - loss: 0.5720 - accuracy: 0.8736 - val_loss: 0.8784 - val_accuracy: 0.8563

Epoch 00006: val_loss improved from 1.06010 to 0.87837, saving model to mymodel_

In [ ]:
## Test prediction

In [ ]:
filename_features_test = 'test_features.csv'
test_seqs = clean_seq(filename_features_test)
test_sequences = tokenizer.texts_to_sequences(test_seqs)

predictions = np.argmax(model.predict(test_sequences), axis=1)
print(predictions)

def decode_labels(encoded_predict_labels, le):
    test_predictions = le.inverse_transform(encoded_predict_labels)
    
    return test_predictions

test_seq_predictions = decode_labels(predictions, le)
print(test_seq_predictions)

[ 659    7  200 ...   77 1059  590]
[ 667    8  205 ...   81 1071  597]


In [ ]:
def get_seq_ids(filename):
    df = pd.read_csv(filename)
    ids = df['id']
    
    return np.array(ids)

test_ids = get_seq_ids(filename_features_test)
print(test_ids)

frames = [pd.DataFrame(test_ids), pd.DataFrame(test_seq_predictions)]
output_data= np.concatenate(frames, axis=1)
output_df = pd.DataFrame(output_data)
output_df.to_csv('dna_barcode_seq_submission11_word_embedding_LSTM.csv', index=False,  header=["id","labels"])

[   1    2    3 ... 8304 8305 8306]


In [ ]:
from google.colab import files
files.download("dna_barcode_seq_submission11_word_embedding_LSTM.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>